# Create single 'pixel' object

- Jacopo Umberto Verga
- 08/13/2024


After pre-processing the images
- [steinbock](https://bodenmillergroup.github.io/steinbock/latest/) extract tiff files
- Extract single channel files with [imagej](https://imagej.net/ij/) and python
- Registering each slide with [skimage](https://scikit-image.org/)

We need to import the pixels values as a "single cell object". The AnnData format allows us to easily handle the pixels and perform spatial analysis as for the 2D data.


Steps:
- Import each pixels as an observation of the AnnData object
    - Channels values as expression, coordinates in the metadata
- Remove zero value pixels (padding)
- Remove non-tissue pixels using watershed segmentation
    - clustering of the pixels with non-tissue pixels increase the noise and masks real biological signal

In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import tifffile as tiff
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import scanpy as sc
from IPython.display import Markdown
from clustering_Modules.utils import image_preprocessing_parallel_dev  as img_pre
from tqdm import tqdm


In [2]:
slide_path='./samples/single_channel_images/D0246_registered/D02461_section02_001/'

In [3]:
# Get all channels
channels=natsorted([('_'.join(string.split('.')[0].split('_')[1:])
                     .replace('_(c-kit)','')
                     .replace('CD73_','')
                     .replace('CD44_','')
                     .replace('TIGIT_','TIGIT')) for string in os.listdir(slide_path)])

In [4]:
def load_image_channels(image_path, channels):
    channel_data = {}
    for channel in channels:
        channel_file=[f for f in os.listdir(slide_path) if channel in f][0]
        channel_filename = os.path.join(image_path, f"{channel_file}")
        # read channel file
        channel_data[channel] = tiff.imread(channel_filename)
    return channel_data


In [5]:
def create_anndata(image_name, channel_data,slide):
    # Extract the dimensions from one of the channels
    example_channel = list(channel_data.values())[0]
    height, width = example_channel.shape
    library_id=image_name + '_' + str(slide)
    # Create a DataFrame for the expression matrix (adata.X)
    expression_matrix = np.zeros((height * width, len(channel_data)))
    # Get expression of each channel as flat array
    for i, (channel, data) in enumerate(channel_data.items()):
        expression_matrix[:, i] = data.flatten()

    img_list = list(channel_data.values())
    img_array = np.stack(img_list, axis=-1)

    # Create the metadata DataFrame (adata.obs)
    x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))
    obs = pd.DataFrame({
        'x': x_coords.flatten(),
        'y': y_coords.flatten(),
        'width_px':width, 
        'height_px':height, 
        'z': slide,
        'image': image_name,
        'library_id':library_id
    })
    # Create spatial dictionary
    uns = {
        library_id:{
            'images':{
                'hires':img_array
            }
        }
    }
    
    # Create the AnnData object
    adata = ad.AnnData(X=expression_matrix, obs=obs, var=pd.DataFrame(index=channel_data.keys()))
    adata.obs_names = adata.obs['x'].astype(str)+'_'+adata.obs['y'].astype(str)+'_'+adata.obs['z'].astype(str)+'_'+image_name
    return adata, uns

In [6]:
all_slides_path='./samples/single_channel_images/'
images_ids=['D0246','G4090','K0401']

In [7]:
image_name=images_ids[0]
def process_sections(all_slides_path, image_name,channels):
    # process all sections of one sample
    image_path=os.path.join(all_slides_path, image_name+'_registered')
    sections=natsorted(os.listdir(image_path))
    adatas=[]
    unses={}
    for i,section in enumerate(sections):
        slide_path=os.path.join(image_path,section)
        channel_data=load_image_channels(slide_path, channels)
        adata,uns=create_anndata(image_name, channel_data,i+1)
        adatas.append(adata)
        unses[list(uns.keys())[0]]=list(uns.values())[0]
    return adatas,unses

In [8]:
adatas=[]
unses={}
for image_name in images_ids:
    # process all images
    adata,uns=process_sections(all_slides_path, image_name,channels)
    adatas.extend(adata)
    unses.update(uns)


In [9]:
# concat adatas of each image
adatas=ad.concat(adatas)

In [10]:
# add .uns['spatial']
adatas.uns['spatial']=unses

In [11]:
# del unses since takes a lot of space
del unses
gc.collect()

In [12]:
# Remove pixels with zero values for all the markers (padding pixels)
sc.pp.filter_cells(adatas,min_genes=1)
adatas

In [14]:
# Create normalized expression layer, change normalization to 99.9 quantile since some markers are really lowly expressed in some slides
adatas.layers['exprs'] = img_pre.normalize_channels_mat(
    img_pre.normalize_channels_mat(
        adatas.X, method='arcsinh', cofactor=None
    ),
    method='quantile', quantile=.999
)


In [15]:
adatas

In [16]:
# Save dirty dataset
adatas.write('./samples/adata_dirty.h5ad')

In [ ]:
del adatas
gc.collect()

In [17]:
adatas=sc.read('./samples/adata_dirty.h5ad')

## Watershed Segmentation:

To select the pixels falling in the tissue I am going to use the function already wrote in adipo_finder, steps for each slide:
- Normalize and sum values from all the channels
- Apply gaussian blur to get smoother borders and reduce noise
- Find markers (areas with high vales)
- Apply watershed segmentation
- Remove small objects, likely to be noise

Then, for each segmented image:
- Add it in .uns['spatial'] as segmentation
- Extract coordinates of segmented pixels and clean the adata object

In [25]:
import numpy as np
from adipo_finder import segmentation
from adipo_finder import utils
from joblib import Parallel, delayed
# Wrapper function for normalization
def normalize_function(mat):
    mat=img_pre.normalize_channels_mat(mat, 
                                       method='arcsinh', 
                                       cofactor=None)
    mat=img_pre.normalize_channels_mat(mat,method='quantile',quantile=.999)
    return mat

def parallel_normalization(img,n_jobs=-1):
    # Process observations in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(normalize_function)(img[:,:,i]) for i in range(img.shape[2])
    )
    # sum normalized channels to increase signal
    mat=np.sum(np.dstack(results),axis=2)
    # Apply filter to reduce noise
    mat=np.where(mat>20,mat,0)
    return mat

def sequential_normalization(img):
    # Process observations sequentially
    results=[]
    for i in range(img.shape[2]):
        results.append(normalize_function(img[:,:,i]))
    # sum normalized channels to increase signal
    mat=np.sum(np.dstack(results),axis=2)
    # apply filter to reduce noise
    mat=np.where(mat>20,mat,0)
    return mat



def run_segmentation(mat,library_id, sigma, window=1, min_size=100):
    mat=utils.Preprocessing.apply_gaussian_filter(mat,sigma)
    distance,markers=segmentation.Segmentation.find_local_maxima(mat)
    segmented_image=segmentation.Segmentation.apply_watershed_segmentation(mat, markers, distance, window)
    segmented_image=segmentation.Segmentation.filter_objects_by_size(segmented_image, min_size=min_size)
    return {library_id: segmented_image}


def parallel_segmentation(adatas,n_jobs=-1,sigma=.5):
    # get all libraries
    libraries = adatas.obs['library_id'].unique()
    images={}
    # First step, normalize in parallel al the channels for each image
    for library_id in tqdm(libraries):
        img=adatas.uns['spatial'][library_id]['images']['hires']
        images[library_id]=parallel_normalization(img)
    print('Normalization done\nStarting parallel segmentation')
    # Second step proceed with segmentation of each image
    results = Parallel(n_jobs=n_jobs)(
        delayed(run_segmentation)(v,k,sigma) for k,v in images.items())
    return results

In [26]:
results=parallel_segmentation(adatas,n_jobs=-1,sigma=1)

100%|██████████| 31/31 [01:40<00:00,  3.24s/it]


Normalization done
Starting parallel segmentation


In [27]:
results={k: v for d in results for k, v in d.items()}

In [124]:
def filter_adata_by_library_and_coordinates(adatas, results):
    # empty to populate with the filtering masks
    filtered_mask=[]
    for k,v in tqdm(results.items()):
        # pre filter adata for the specific library_id speed up the process
        adata=adatas[adatas.obs['library_id']==k]
        # get x,y coordinates of nonzero pixels
        y,x=v.nonzero()
        # create list of x_y coordinates
        filter_coords=list(map('_'.join,zip(x.astype(str),y.astype(str))))
        # filtering mask for library_id
        current_mask = adata.obs['coords'].isin(filter_coords)
        # Add to the list of masks
        filtered_mask.append(current_mask)
        # add segmentation mask to adatas
        adatas.uns['spatial'][k]['images']['segmentation']=v
    # concat filtering masks
    filtered_mask = pd.concat(filtered_mask)
    # filter original
    adata_filtered = adatas[filtered_mask].copy()
    
    return adata_filtered


In [125]:
adatas.obs['coords'] = adatas.obs['x'].astype(str) + '_' +adatas.obs['y'].astype(str)

In [126]:
adatas_clean=filter_adata_by_library_and_coordinates(adatas,results)

100%|██████████| 31/31 [00:29<00:00,  1.05it/s]


In [123]:
adatas_clean

AnnData object with n_obs × n_vars = 3488099 × 39
    obs: 'x', 'y', 'width_px', 'height_px', 'z', 'image', 'library_id', 'n_genes', 'coords'
    uns: 'spatial'
    layers: 'exprs'

## Slides plot

Plotting registered images, segmentation masks, kept pixels and original photos to verify results

In [147]:

def plot_slides(adatas,library_id, markers=['ICSK1','ICSK2']):
    adata=adatas[adatas.obs['library_id']==library_id]
    x_axis=2
    y_axis=2
    fig,axes=plt.subplots(2, 2, figsize=(10,10))
    axes = axes.flatten()
    ## Plot original image with summed channels
    img=np.sum(adata.uns['spatial'][library_id]['images']['hires'],axis=2)
    axes[0].imshow(img)
    axes[0].set_title('Registered Image')
    ## Plot segmentation image
    axes[1].imshow(adata.uns['spatial'][library_id]['images']['segmentation'])
    axes[1].set_title('Segmentation Image')
    ## Plot points
    axes[2].scatter(adata.obs['x'],
                    adata.obs['y'],
                    s=.1,
                    c=sc.get.obs_df(adata,markers,
                                    layer='exprs').sum(axis=1)
                   )
    axes[2].set_xlim(0,adata.obs['width_px'].unique())
    axes[2].set_ylim(adata.obs['height_px'].unique(),0)
    axes[2].set_aspect(adata.obs['height_px'].unique()/adata.obs['width_px'].unique())
    axes[2].set_title('Scatterplot trasformed pixels')
    ## Plot photo
    png=plt.imread(f"png_images/{adata.obs['image'].unique()[0]}_section{adata.obs['z'].unique()[0]}.png")
    axes[3].imshow(png)
    axes[3].set_title('Photo')
    fig.suptitle(library_id)
    plt.tight_layout()
    plt.show()


sample=images_ids[0]
Markdown(f"""
### Sample: {sample}
""")


### Sample: D0246


In [ ]:
libraries=adatas[adatas.obs['image']==sample].obs['library_id'].unique()
for library_id in libraries:
    plot_slides(adatas_clean,library_id, markers=list(adatas.var_names))

In [ ]:
sample=images_ids[1]
Markdown(f"""
### Sample: {sample}
""")

In [ ]:
libraries=adatas[adatas.obs['image']==sample].obs['library_id'].unique()
for library_id in libraries:
    plot_slides(adatas_clean,library_id, markers=list(adatas.var_names))

In [ ]:
sample=images_ids[2]
Markdown(f"""
### Sample: {sample}
""")

In [ ]:
libraries=adatas[adatas.obs['image']==sample].obs['library_id'].unique()
for library_id in libraries:
    plot_slides(adatas_clean,library_id, markers=list(adatas.var_names))

## Save clean adata

In [ ]:
adatas_clean.write('./samples/adata_clean.h5ad')